# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set our OpenAI, Tavily, and LangSmith API keys along with our LangSmith environment variables.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [4]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [46]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE8 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [8]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

##### ✅ Answer:

It happens when we run `model = model.bind_tools(tool_belt)`.

The LangChain framework converts each tool in the tool belt into an OpenAI function calling format.

Essentially it is taking the tool's name, description, and parameter schema. This is converted inot a JSON schema that is then attachend/sent along with every request to the LLM.

The tool's description is the most important, essentially describing how it works and when it should be used. The  the LLM decides to use it or not based on these details.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [9]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [10]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [11]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [12]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [13]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [14]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [15]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

##### ✅ Answer:

Yes, the default is 25 steps. I found it in the docs: https://langchain-ai.github.io/langgraph/concepts/low_level/#recursion-limit

If not, how could we impose a limit to the number of cycles?

##### ✅ Answer:

We could change the defualt limit to whatever went want by adding to the `config` param of the `invoke` method like so:
config={"recursion_limit": 5}



## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [25]:
from langchain_core.messages import HumanMessage

# inputs = {"messages" : [HumanMessage(content="How are technical professionals using AI to improve their work?")]}

# I needed a more precise questions where it felt it needed to use the tools.
inputs = {"messages" : [HumanMessage(content="What are the latest AI trends in 2025?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CbMADH6W38yKA7Czmph7wpp', 'function': {'arguments': '{"query":"latest AI trends in 2025"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 163, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ71iBKFAh40XYQxxHFMvOXSPBaax', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9958d409-62b8-445d-ba76-f19b86f52670-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'latest AI trends in 2025'}, 'id': 'call_4CbMADH6W38yKA7Czmph7wpp', 'type': 'tool_call'}]

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [26]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the A Comprehensive Survey of Deep Research paper, then search each of the authors to find out where they work now using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_givL2GsCYvC7NZj2J8Hl1yj1', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_35y5ynwQkKmguvhgZwoKjh98', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research paper"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 182, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ73An3L0hHZQOPvcuQuXGfhjd3ZW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4515e093-6840-42

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

##### ✅ Answer:

Agent:
- From the tools it realzied it needs to use both Arxiv and Tavily tools
- Made calls to both with the query "A Comprehensive Survey of Deep Research paper"

Action:
- Arxiv returned A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications
- Tavily gave more details and confirmation

Agent:
- found authors
- new calls to tavily for the 2 authors

Action:
- Retrieved current employment information for both authors

Agent:
- Compiled the findigs to a response
2 full cycles (agent / action / agent / action / agent)



Specifically my agent did the following:


Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_givL2GsCYvC7NZj2J8Hl1yj1', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_35y5ynwQkKmguvhgZwoKjh98', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research paper"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 182, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ73An3L0hHZQOPvcuQuXGfhjd3ZW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4515e093-6840-42c1-a63e-ba1dc6d2728c-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'A Comprehensive Survey of Deep Research'}, 'id': 'call_givL2GsCYvC7NZj2J8Hl1yj1', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': 'A Comprehensive Survey of Deep Research paper'}, 'id': 'call_35y5ynwQkKmguvhgZwoKjh98', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 59, 'total_tokens': 241, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]



Receiving update from node: 'action'
Tool Used: arxiv
[ToolMessage(content='Published: 2025-06-14\nTitle: A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications\nAuthors: Renjun Xu, Jingwen Peng\nSummary: This survey examines the rapidly evolving field of Deep Research systems --\nAI-powered applications that automate complex research workflows through the\nintegration of large language models, advanced information retrieval, and\nautonomous reasoning capabilities. We analyze more than 80 commercial and\nnon-commercial implementations that have emerged since 2023, including\nOpenAI/Deep Research, Gemini/Deep Research, Perplexity/Deep Research, and\nnumerous open-source alternatives. Through comprehensive examination, we\npropose a novel hierarchical taxonomy that categorizes systems according to\nfour fundamental technical dimensions: foundation models and reasoning engines,\ntool utilization and environmental interaction, task planning and execution\ncontrol, and knowledge synthesis and output generation. We explore the\narchitectural patterns, implementation approaches, and domain-specific\nadaptations that characterize these systems across academic, scientific,\nbusiness, and educational applications. Our analysis reveals both the\nsignificant capabilities of current implementations and the technical and\nethical challenges they present regarding information accuracy, privacy,\nintellectual property, and accessibility. The survey concludes by identifying\npromising research directions in advanced reasoning architectures, multimodal\nintegration, domain specialization, human-AI collaboration, and ecosystem\nstandardization that will likely shape the future evolution of this\ntransformative technology. By providing a comprehensive framework for\nunderstanding Deep Research systems, this survey contributes to both the\ntheoretical understanding of AI-augmented knowledge work and the practical\ndevelopment of more capable, responsible, and accessible research technologies.\nThe paper resources can be viewed at\nhttps://github.com/scienceaix/deepresearch.\n\nPublished: 2021-03-05\nTitle: A comprehensive survey on point cloud registration\nAuthors: Xiaoshui Huang, Guofeng Mei, Jian Zhang, Rana Abbas\nSummary: Registration is a transformation estimation problem between two point clouds,\nwhich has a unique and critical role in numerous computer vision applications.\nThe developments of optimization-based methods and deep learning methods have\nimproved registration robustness and efficiency. Recently, the combinations of\noptimization-based and deep learning methods have further improved performance.\nHowever, the connections between optimization-based and deep learning methods\nare still unclear. Moreover, with the recent development of 3D sensors and 3D\nreconstruction techniques, a new research direction emerges to align\ncross-source point clouds. This survey conducts a comprehensive survey,\nincluding both same-source and cross-source registration methods, and summarize\nthe connections between optimization-based and deep learning methods, to\nprovide further research insight. This survey also builds a new benchmark to\nevaluate the state-of-the-art registration algorithms in solving cross-source\nchallenges. Besides, this survey summarizes the benchmark data sets and\ndiscusses point cloud registration applications across various domains.\nFinally, this survey proposes potential research directions in this rapidly\ngrowing field.\n\nPublished: 2023-07-07\nTitle: A Survey of Deep Learning in Sports Applications: Perception, Comprehension, and Decision\nAuthors: Zhonghan Zhao, Wenhao Chai, Shengyu Hao, Wenhao Hu, Guanhong Wang, Shidong Cao, Mingli Song, Jenq-Neng Hwang, Gaoang Wang\nSummary: Deep learning has the potential to revolutionize sports performance, with\napplications ranging from perception and comprehension to decision. This paper\npresents a comprehensive survey of deep learning in sports performance,\nfocusing on three main aspects: algorithms, datasets and virtual environments,\nand challenges. Firstly, we discuss th', name='arxiv', id='c2f48b19-1489-4256-b33b-18b5333d3754', tool_call_id='call_givL2GsCYvC7NZj2J8Hl1yj1'), ToolMessage(content='[{"title": "[2506.12594] A Comprehensive Survey of Deep Research - arXiv", "url": "https://arxiv.org/abs/2506.12594", "content": "|  |  |\\n --- |\\n| Comments: | 95 pages, 11 figures |\\n| Subjects: | Artificial Intelligence (cs.AI); Multiagent Systems (cs.MA) |\\n| ACM classes: | I.2.8 |\\n| Cite as: | arXiv:2506.12594 [cs.AI] |\\n|  | (or  arXiv:2506.12594v1 [cs.AI] for this version) |\\n|  |  arXiv-issued DOI via DataCite |\\n\\n## Submission history\\n\\nFrom: Renjun Xu [view email] [v1] Sat, 14 Jun 2025 18:19:05 UTC (652 KB)\\n\\nFull-text links:\\n\\n## Access Paper: [...] We gratefully acknowledge support from the Simons Foundation, member institutions, and all contributors. Donate\\n\\n> cs > arXiv:2506.12594\\n\\n# Computer Science > Artificial Intelligence\\n\\narXiv:2506.12594 (cs)\\n\\n[Submitted on 14 Jun 2025]\\n\\n# Title:A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications\\n\\nAuthors:Renjun Xu, Jingwen Peng [...] > Abstract:This survey examines the rapidly evolving field of Deep Research systems -- AI-powered applications that automate complex research workflows through the integration of large language models, advanced information retrieval, and autonomous reasoning capabilities. We analyze more than 80 commercial and non-commercial implementations that have emerged since 2023, including OpenAI/Deep Research, Gemini/Deep Research, Perplexity/Deep Research, and numerous open-source alternatives. Through", "score": 0.88822687}, {"title": "[PDF] A Comprehensive Survey of Deep Research - arXiv", "url": "https://arxiv.org/pdf/2506.12594?", "content": "Deep Research refers to the systematic application of AI technologies to automate and enhance research processes through three core dimensions: (1) Intelligent Knowledge Discovery: Automating literature search, hypothesis generation, and pattern recognition across heterogeneous data sources (2) End-to-End Workflow Automation: Integrating experimental design, data collection, analysis, and result interpretation into unified AI-driven pipelines (3) Collaborative Intelligence Enhancement: [...] This survey addresses three fundamental questions: (1) How do architectural choices (system architecture, implementation approach, functional capabilities) impact Deep Research effectiveness? (2) What technical innovations have emerged in LLM fine-tuning, retrieval mechanisms, and workflow orchestration across the spectrum of Deep Research implementations? (3) How do existing systems balance performance, usability, and ethical considerations, and what patterns emerge from comparing approaches [...] development, with specific attention to emerging architectures and integration opportunities The remainder of this paper follows a structured exploration beginning with conceptual frameworks (Section 2), technical innovations and comparative analysis (Sections 3-4), implementation technologies (Section 5), evaluation methodologies (Section 6), applications and use cases (Section 7), ethical considerations (Section 8), and future directions (Section 9).", "score": 0.8033131}, {"title": "A comprehensive survey of deep learning research on medical ...", "url": "https://www.sciencedirect.com/science/article/abs/pii/S0899707122002856", "content": "Skip to main contentSkip to article\\n\\nSign in\\n\\n Access through your organization\\n Purchase PDF\\n Patient Access\\n\\nWe use cookies that are necessary to make our site work. We may also use additional cookies to analyze, improve, and personalize our content and your digital experience. You can manage your cookie preferences using the “Cookie Settings” link. For more information, see ourCookie Policy\\n\\n## Cookie Preference Center [...] We use cookies which are necessary to make our site work. We may also use additional cookies to analyse, improve and personalise our content and your digital experience. For more information, see our Cookie Policy and the list of Google Ad-Tech Vendors. [...] You may choose not to allow some types of cookies. However, blocking some types may impact your experience of our site and the services we are able to offer. See the different category headings below to find out more or change your settings.\\n  \\n  \\nYou may also be able to exercise your privacy choices as described in our Privacy Policy", "score": 0.5521781}, {"title": "Top Research Papers on Deep Learning - Must Read List", "url": "https://paperguide.ai/papers/top/research-papers-deep-learning/", "content": "This is the first comprehensive survey in the efficient deep learning space that covers the landscape of model efficiency from modeling techniques to hardware support and the seminal work there.\\n\\nChat\\n\\nCite\\n\\nReferences\\n\\n### Deep Reading, Deep Learning\\n2 Citations 2022\\n\\nauthors unavailable\\n\\njournal unavailable [...] This paper surveys comprehensively recent advances in deep class-incremental learning and summarizes these methods from three aspects, i.e. , data-centric, model-centric, and algorithm-centric, and provides a rigorous and uniﬁed evaluation of 16 methods in benchmark image class-incremental learning tasks to find out the characteristics of different algorithms empirically.\\n\\nChat\\n\\nCite\\n\\nReferences\\n\\n### Deep Learning with Logical Constraints\\n62 Citations 2022 [...] IEEE Access\\n\\nThis paper comprehensively discussed the attacks and defenses in four popular DL models, including DNN, DRL, FL, and TL, and highlighted the application domains, datasets, metrics, and testbeds in these fields.\\n\\nChat\\n\\nCite\\n\\nReferences\\n\\n### Deep Class-Incremental Learning: A Survey\\n103 Citations 2023\\n\\nDa-Wei Zhou, Qiwen Wang, Zhi-Hong Qi + 3 more\\n\\nArXiv", "score": 0.46337196}, {"title": "Deep Research Tools: A Comprehensive Guide and Comparison", "url": "https://bytebridge.medium.com/deep-research-tools-a-comprehensive-guide-and-comparison-a38077d52122", "content": "refine your query or add filters (like specifying a date range or field of study) and it will update the results near-instantly. The model’s summarization for each paper is done on the fly but it’s quite fast. One reason it’s speedy is that it doesn’t generate a huge narrative, it’s pulling bite-sized info (which is easier and quicker for an AI to produce). Also, much of the heavy lifting (searching papers) is done through efficient information retrieval systems. In short, among our list, [...] ## Report Quality (Depth, Structure, Accuracy, Relevance)\\n\\nOne of the biggest considerations when choosing a deep research tool is the quality of the report or output it generates. Does it produce in-depth content or just superficial summaries? Are its reports well-structured and coherent? Does it stay accurate and cite relevant sources? Here’s how each tool fares: [...] content, it might miss non-scholarly but relevant information (news, statistics from reports, etc.).", "score": 0.45551404}]', name='tavily_search_results_json', id='ae6e0553-c3ce-477a-9db1-d67c035ca66a', tool_call_id='call_35y5ynwQkKmguvhgZwoKjh98', artifact={'query': 'A Comprehensive Survey of Deep Research paper', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://arxiv.org/abs/2506.12594', 'title': '[2506.12594] A Comprehensive Survey of Deep Research - arXiv', 'content': '|  |  |\n --- |\n| Comments: | 95 pages, 11 figures |\n| Subjects: | Artificial Intelligence (cs.AI); Multiagent Systems (cs.MA) |\n| ACM classes: | I.2.8 |\n| Cite as: | arXiv:2506.12594 [cs.AI] |\n|  | (or  arXiv:2506.12594v1 [cs.AI] for this version) |\n|  |  arXiv-issued DOI via DataCite |\n\n## Submission history\n\nFrom: Renjun Xu [view email] [v1] Sat, 14 Jun 2025 18:19:05 UTC (652 KB)\n\nFull-text links:\n\n## Access Paper: [...] We gratefully acknowledge support from the Simons Foundation, member institutions, and all contributors. Donate\n\n> cs > arXiv:2506.12594\n\n# Computer Science > Artificial Intelligence\n\narXiv:2506.12594 (cs)\n\n[Submitted on 14 Jun 2025]\n\n# Title:A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications\n\nAuthors:Renjun Xu, Jingwen Peng [...] > Abstract:This survey examines the rapidly evolving field of Deep Research systems -- AI-powered applications that automate complex research workflows through the integration of large language models, advanced information retrieval, and autonomous reasoning capabilities. We analyze more than 80 commercial and non-commercial implementations that have emerged since 2023, including OpenAI/Deep Research, Gemini/Deep Research, Perplexity/Deep Research, and numerous open-source alternatives. Through', 'score': 0.88822687, 'raw_content': None}, {'url': 'https://arxiv.org/pdf/2506.12594?', 'title': '[PDF] A Comprehensive Survey of Deep Research - arXiv', 'content': 'Deep Research refers to the systematic application of AI technologies to automate and enhance research processes through three core dimensions: (1) Intelligent Knowledge Discovery: Automating literature search, hypothesis generation, and pattern recognition across heterogeneous data sources (2) End-to-End Workflow Automation: Integrating experimental design, data collection, analysis, and result interpretation into unified AI-driven pipelines (3) Collaborative Intelligence Enhancement: [...] This survey addresses three fundamental questions: (1) How do architectural choices (system architecture, implementation approach, functional capabilities) impact Deep Research effectiveness? (2) What technical innovations have emerged in LLM fine-tuning, retrieval mechanisms, and workflow orchestration across the spectrum of Deep Research implementations? (3) How do existing systems balance performance, usability, and ethical considerations, and what patterns emerge from comparing approaches [...] development, with specific attention to emerging architectures and integration opportunities The remainder of this paper follows a structured exploration beginning with conceptual frameworks (Section 2), technical innovations and comparative analysis (Sections 3-4), implementation technologies (Section 5), evaluation methodologies (Section 6), applications and use cases (Section 7), ethical considerations (Section 8), and future directions (Section 9).', 'score': 0.8033131, 'raw_content': None}, {'url': 'https://www.sciencedirect.com/science/article/abs/pii/S0899707122002856', 'title': 'A comprehensive survey of deep learning research on medical ...', 'content': 'Skip to main contentSkip to article\n\nSign in\n\n Access through your organization\n Purchase PDF\n Patient Access\n\nWe use cookies that are necessary to make our site work. We may also use additional cookies to analyze, improve, and personalize our content and your digital experience. You can manage your cookie preferences using the “Cookie Settings” link. For more information, see ourCookie Policy\n\n## Cookie Preference Center [...] We use cookies which are necessary to make our site work. We may also use additional cookies to analyse, improve and personalise our content and your digital experience. For more information, see our Cookie Policy and the list of Google Ad-Tech Vendors. [...] You may choose not to allow some types of cookies. However, blocking some types may impact your experience of our site and the services we are able to offer. See the different category headings below to find out more or change your settings.\n  \n  \nYou may also be able to exercise your privacy choices as described in our Privacy Policy', 'score': 0.5521781, 'raw_content': None}, {'url': 'https://paperguide.ai/papers/top/research-papers-deep-learning/', 'title': 'Top Research Papers on Deep Learning - Must Read List', 'content': 'This is the first comprehensive survey in the efficient deep learning space that covers the landscape of model efficiency from modeling techniques to hardware support and the seminal work there.\n\nChat\n\nCite\n\nReferences\n\n### Deep Reading, Deep Learning\n2 Citations 2022\n\nauthors unavailable\n\njournal unavailable [...] This paper surveys comprehensively recent advances in deep class-incremental learning and summarizes these methods from three aspects, i.e. , data-centric, model-centric, and algorithm-centric, and provides a rigorous and uniﬁed evaluation of 16 methods in benchmark image class-incremental learning tasks to find out the characteristics of different algorithms empirically.\n\nChat\n\nCite\n\nReferences\n\n### Deep Learning with Logical Constraints\n62 Citations 2022 [...] IEEE Access\n\nThis paper comprehensively discussed the attacks and defenses in four popular DL models, including DNN, DRL, FL, and TL, and highlighted the application domains, datasets, metrics, and testbeds in these fields.\n\nChat\n\nCite\n\nReferences\n\n### Deep Class-Incremental Learning: A Survey\n103 Citations 2023\n\nDa-Wei Zhou, Qiwen Wang, Zhi-Hong Qi + 3 more\n\nArXiv', 'score': 0.46337196, 'raw_content': None}, {'url': 'https://bytebridge.medium.com/deep-research-tools-a-comprehensive-guide-and-comparison-a38077d52122', 'title': 'Deep Research Tools: A Comprehensive Guide and Comparison', 'content': 'refine your query or add filters (like specifying a date range or field of study) and it will update the results near-instantly. The model’s summarization for each paper is done on the fly but it’s quite fast. One reason it’s speedy is that it doesn’t generate a huge narrative, it’s pulling bite-sized info (which is easier and quicker for an AI to produce). Also, much of the heavy lifting (searching papers) is done through efficient information retrieval systems. In short, among our list, [...] ## Report Quality (Depth, Structure, Accuracy, Relevance)\n\nOne of the biggest considerations when choosing a deep research tool is the quality of the report or output it generates. Does it produce in-depth content or just superficial summaries? Are its reports well-structured and coherent? Does it stay accurate and cite relevant sources? Here’s how each tool fares: [...] content, it might miss non-scholarly but relevant information (news, statistics from reports, etc.).', 'score': 0.45551404, 'raw_content': None}], 'response_time': 1.63, 'request_id': '41cc526b-2883-4ac7-b1b3-3a7b9a670c5b'})]



Receiving update from node: 'agent'
[AIMessage(content='The paper titled "A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications" was published on June 14, 2025, authored by Renjun Xu and Jingwen Peng. \n\nI will now proceed to find out where the authors currently work.', additional_kwargs={'tool_calls': [{'id': 'call_ObzZFG5aVf8NAxQ058eg2d8h', 'function': {'arguments': '{"query": "Renjun Xu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_KuvSnvV0DYPgSAFZMnxAqYLV', 'function': {'arguments': '{"query": "Jingwen Peng"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 2574, 'total_tokens': 2686, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ73FKxf4Lf7DrH1et8k043TYbhAd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8fb80e27-78e4-421a-9c5e-9f702ae7cc23-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Renjun Xu'}, 'id': 'call_ObzZFG5aVf8NAxQ058eg2d8h', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': 'Jingwen Peng'}, 'id': 'call_KuvSnvV0DYPgSAFZMnxAqYLV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2574, 'output_tokens': 112, 'total_tokens': 2686, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]



Receiving update from node: 'action'
Tool Used: tavily_search_results_json
[ToolMessage(content='[{"title": "Renjun Xu - Researcher, Zhejiang University - OpenReview", "url": "https://openreview.net/profile?id=~Renjun_Xu1", "content": "# Renjun Xu\\n\\n### Principal Researcher, Zhejiang University\\n\\n#### Names\\n\\n#### Emails\\n\\n#### Personal Links\\n\\n#### Career & Education History\\n\\n#### Advisors, Relations & Conflicts\\n\\nNo relations added\\n\\n#### Expertise\\n\\n#### Publications\\n\\n#### scKGOT: Intercellular Signaling Inference with Knowledge Graph Optimal Transport for Single-cell Transcriptomics)\\n\\n#### $E(2)$-Equivariant Vision Transformer)\\n\\n#### Critical Temperature Prediction of Superconductors Based on Machine Learning: A Short Review) [...] #### Exploiting Adapters for Cross-Lingual Low-Resource Speech Recognition)\\n\\n#### Hierarchical knowledge amalgamation with dual discriminative feature alignment)\\n\\n#### Modeling Dynamic Missingness of Implicit Feedback for Sequential Recommendation)\\n\\n#### S2SNet: A Pretrained Neural Network for Superconductivity Discovery)\\n\\n#### Learning Interest-oriented Universal User Representation via Self-supervision) [...] #### Learning Universal User Representations via Self-Supervised Lifelong Behaviors Modeling)\\n\\n#### Learning Invariant Representations across Domains and Tasks)\\n\\n#### Co-Authors\\n\\nOpenReview is a long-term project to advance science through improved peer review with legal nonprofit status. We gratefully acknowledge the support of the OpenReview Sponsors. © 2025 OpenReview", "score": 0.6410185}, {"title": "Renjun Xu - Center for Data Science, Zhejiang University | 人才画像", "url": "https://www.aminer.cn/profile/renjun-xu/53f42ceddabfaedd74d30355?source=bz1", "content": "Renjun Xu - Center for Data Science, Zhejiang University | 人才画像 - AMiner\\n\\n\\n\\nResearch\\n\\nCenter for Data Science Zhejiang University\\n\\n、《International Joint Conference on Artificial Intelligence》(IJCAI, CCF-A), 《IEEE Transactions on Knowledge and Data Engineering》(TKDE, CCF-A)交叉领域发表多篇国际顶尖期刊和会议文章，CVPR、AAAI、NIPS、TPAMI、TIP、TLT等顶级人工智能期刊和会议程序委员会委员，荣获2020年度世界人工智能大会青年优秀论文提名奖，指导并推荐的所有学生均已拿到麻省理工学院(MIT)、卡内基梅隆大学(CMU)等全球顶尖名校的offer！\\n\\nEducation\\n\\nSign in to view more\\n\\nExperience\\n\\nSign in to view more [...] Research Interests\\n\\n2012 2025\\n\\nPapers 共 39 篇 Patents 共 9 篇 Author Statistics Co-Author Similar Experts\\n\\nBy Year By Citation 主题筛选 期刊级别筛选 合作者筛选 合作机构筛选\\n\\n时间\\n\\n引用量\\n\\n主题\\n\\n期刊级别\\n\\n合作者\\n\\n合作机构\\n\\nAll 2025 2024 2023 2022 2021 2020 2015 2014 2013 2012 2010 2006\\n\\nDo PhD-level LLMs Truly Grasp Elementary Addition? Probing Rule Learning Vs. Memorization in Large Language Models\\n\\nYang Yan,Yu Lu,Renjun Xu,Zhenzhong Lan\\n\\narXiv · Computation and Language（2025）\\n\\nCited 0 Views 11 Bibtex\\n\\n0\\n\\n11 [...] The page data are from open Internet sources, cooperative publishers and automatic analysis results through AI technology. We do not make any commitments and guarantees for the validity, accuracy, correctness, reliability, completeness and timeliness of the page data. If you have any questions, please contact us by email: report@aminer.cn\\n\\nSwipe to Fine Result", "score": 0.60049355}, {"title": "Renjun Hu\'s Homepage", "url": "https://hurenjun.github.io/", "content": "an algorithm engineer at Alibaba Cloud, contributing to AI-driven transformations across various business\\ndomains including feed recommendation, user growth, online marketing, and LLM-as-a-Judge.\\nSince January 2025, he has joined the School of Data Science and Engineering, East China Normal University\\nas a young researcher.\\nHis recent research interests include robust machine learning and the understanding, evaluation, and applications of large language models. [...] Renjun Hu received his Bachelor\'s degree in 2014 and Ph.D. in 2020 from the School of Computer Science\\nand Engineering at Beihang University. From September 2017 to April 2018, he was a joint Ph.D. student\\nin the Data Mining Group at Rutgers University. He then worked as a research intern at the Business\\nIntelligence Lab of Baidu Research from May 2018 to September 2019. During 2020 to 2024, he served as [...] Renjun\'s avatar\\n\\n### Renjun Hu     (胡仁君)\\n\\n#### Young Researcher\\n\\nSchool of Data Science of Engineering (DaSE)  \\nEast China Normal University (ECNU)\\n\\n \\nRoom X109, Shuxueguan, Putuo Campus\\n\\n \\nrjhu [at] dase.ecnu.edu.cn    renjun0hu [at] gmail.com\\n\\n### Short Bio", "score": 0.59880555}, {"title": "Renjun XU | UCD | Department of Physics | Research profile", "url": "https://www.researchgate.net/profile/Renjun-Xu", "content": "Skills and Expertise: Electronic Structure. Current institution: University of California, Davis. University of California, Davis.", "score": 0.5683445}, {"title": "Xu Genjun - Wikipedia", "url": "https://en.wikipedia.org/wiki/Xu_Genjun", "content": "Xu was born in She County, Anhui on 23 November 1935. He graduated from the Chemistry Department of Fudan University in 1957, and joined the Shanghai Institute of Biochemistry afterwards, where he participated in and made significant contributions to the insulin synthesis project led by Wang Yinglai. He later researched enzyme kinetics, chemical modification of proteins, the relationship between proteins\' structure and function, and the folding and refolding of proteins. [...] Xu published more than 100 research papers in academic journals. He was a two-time winner of the State Natural Science Award (first class), in addition to several national prizes by the Chinese Academy of Sciences (CAS). He was also awarded the Ho Leung Ho Lee Prize for Life Sciences. He was elected as an academician of the CAS in 1991.\\n\\nXu died on 8 January 2008 at Zhongshan Hospital in Shanghai, at the age of 72.\\n\\n## References\\n\\nWikimedia Foundation\\nPowered by MediaWiki [...] Wikipedia\\nThe Free Encyclopedia\\n\\n## Contents\\n\\n# Xu Genjun\\n\\nXu Genjun (Chinese: 许根俊; 23 November 1935 – 8 January 2008) was a Chinese biochemist. He was a professor at the Shanghai Institute of Biochemistry and Cell Biology. He was an academician of the Chinese Academy of Sciences and President of the Chinese Society of Biochemistry and Molecular Biology.\\n\\n## Biography", "score": 0.25375047}]', name='tavily_search_results_json', id='c5b33580-a980-4716-9472-ee857af9f2cc', tool_call_id='call_ObzZFG5aVf8NAxQ058eg2d8h', artifact={'query': 'Renjun Xu', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://openreview.net/profile?id=~Renjun_Xu1', 'title': 'Renjun Xu - Researcher, Zhejiang University - OpenReview', 'content': '# Renjun Xu\n\n### Principal Researcher, Zhejiang University\n\n#### Names\n\n#### Emails\n\n#### Personal Links\n\n#### Career & Education History\n\n#### Advisors, Relations & Conflicts\n\nNo relations added\n\n#### Expertise\n\n#### Publications\n\n#### scKGOT: Intercellular Signaling Inference with Knowledge Graph Optimal Transport for Single-cell Transcriptomics)\n\n#### $E(2)$-Equivariant Vision Transformer)\n\n#### Critical Temperature Prediction of Superconductors Based on Machine Learning: A Short Review) [...] #### Exploiting Adapters for Cross-Lingual Low-Resource Speech Recognition)\n\n#### Hierarchical knowledge amalgamation with dual discriminative feature alignment)\n\n#### Modeling Dynamic Missingness of Implicit Feedback for Sequential Recommendation)\n\n#### S2SNet: A Pretrained Neural Network for Superconductivity Discovery)\n\n#### Learning Interest-oriented Universal User Representation via Self-supervision) [...] #### Learning Universal User Representations via Self-Supervised Lifelong Behaviors Modeling)\n\n#### Learning Invariant Representations across Domains and Tasks)\n\n#### Co-Authors\n\nOpenReview is a long-term project to advance science through improved peer review with legal nonprofit status. We gratefully acknowledge the support of the OpenReview Sponsors. © 2025 OpenReview', 'score': 0.6410185, 'raw_content': None}, {'url': 'https://www.aminer.cn/profile/renjun-xu/53f42ceddabfaedd74d30355?source=bz1', 'title': 'Renjun Xu - Center for Data Science, Zhejiang University | 人才画像', 'content': 'Renjun Xu - Center for Data Science, Zhejiang University | 人才画像 - AMiner\n\n\n\nResearch\n\nCenter for Data Science Zhejiang University\n\n、《International Joint Conference on Artificial Intelligence》(IJCAI, CCF-A), 《IEEE Transactions on Knowledge and Data Engineering》(TKDE, CCF-A)交叉领域发表多篇国际顶尖期刊和会议文章，CVPR、AAAI、NIPS、TPAMI、TIP、TLT等顶级人工智能期刊和会议程序委员会委员，荣获2020年度世界人工智能大会青年优秀论文提名奖，指导并推荐的所有学生均已拿到麻省理工学院(MIT)、卡内基梅隆大学(CMU)等全球顶尖名校的offer！\n\nEducation\n\nSign in to view more\n\nExperience\n\nSign in to view more [...] Research Interests\n\n2012 2025\n\nPapers 共 39 篇 Patents 共 9 篇 Author Statistics Co-Author Similar Experts\n\nBy Year By Citation 主题筛选 期刊级别筛选 合作者筛选 合作机构筛选\n\n时间\n\n引用量\n\n主题\n\n期刊级别\n\n合作者\n\n合作机构\n\nAll 2025 2024 2023 2022 2021 2020 2015 2014 2013 2012 2010 2006\n\nDo PhD-level LLMs Truly Grasp Elementary Addition? Probing Rule Learning Vs. Memorization in Large Language Models\n\nYang Yan,Yu Lu,Renjun Xu,Zhenzhong Lan\n\narXiv · Computation and Language（2025）\n\nCited 0 Views 11 Bibtex\n\n0\n\n11 [...] The page data are from open Internet sources, cooperative publishers and automatic analysis results through AI technology. We do not make any commitments and guarantees for the validity, accuracy, correctness, reliability, completeness and timeliness of the page data. If you have any questions, please contact us by email: report@aminer.cn\n\nSwipe to Fine Result', 'score': 0.60049355, 'raw_content': None}, {'url': 'https://hurenjun.github.io/', 'title': "Renjun Hu's Homepage", 'content': "an algorithm engineer at Alibaba Cloud, contributing to AI-driven transformations across various business\ndomains including feed recommendation, user growth, online marketing, and LLM-as-a-Judge.\nSince January 2025, he has joined the School of Data Science and Engineering, East China Normal University\nas a young researcher.\nHis recent research interests include robust machine learning and the understanding, evaluation, and applications of large language models. [...] Renjun Hu received his Bachelor's degree in 2014 and Ph.D. in 2020 from the School of Computer Science\nand Engineering at Beihang University. From September 2017 to April 2018, he was a joint Ph.D. student\nin the Data Mining Group at Rutgers University. He then worked as a research intern at the Business\nIntelligence Lab of Baidu Research from May 2018 to September 2019. During 2020 to 2024, he served as [...] Renjun's avatar\n\n### Renjun Hu     (胡仁君)\n\n#### Young Researcher\n\nSchool of Data Science of Engineering (DaSE)  \nEast China Normal University (ECNU)\n\n \nRoom X109, Shuxueguan, Putuo Campus\n\n \nrjhu [at] dase.ecnu.edu.cn    renjun0hu [at] gmail.com\n\n### Short Bio", 'score': 0.59880555, 'raw_content': None}, {'url': 'https://www.researchgate.net/profile/Renjun-Xu', 'title': 'Renjun XU | UCD | Department of Physics | Research profile', 'content': 'Skills and Expertise: Electronic Structure. Current institution: University of California, Davis. University of California, Davis.', 'score': 0.5683445, 'raw_content': None}, {'url': 'https://en.wikipedia.org/wiki/Xu_Genjun', 'title': 'Xu Genjun - Wikipedia', 'content': "Xu was born in She County, Anhui on 23 November 1935. He graduated from the Chemistry Department of Fudan University in 1957, and joined the Shanghai Institute of Biochemistry afterwards, where he participated in and made significant contributions to the insulin synthesis project led by Wang Yinglai. He later researched enzyme kinetics, chemical modification of proteins, the relationship between proteins' structure and function, and the folding and refolding of proteins. [...] Xu published more than 100 research papers in academic journals. He was a two-time winner of the State Natural Science Award (first class), in addition to several national prizes by the Chinese Academy of Sciences (CAS). He was also awarded the Ho Leung Ho Lee Prize for Life Sciences. He was elected as an academician of the CAS in 1991.\n\nXu died on 8 January 2008 at Zhongshan Hospital in Shanghai, at the age of 72.\n\n## References\n\nWikimedia Foundation\nPowered by MediaWiki [...] Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Xu Genjun\n\nXu Genjun (Chinese: 许根俊; 23 November 1935 – 8 January 2008) was a Chinese biochemist. He was a professor at the Shanghai Institute of Biochemistry and Cell Biology. He was an academician of the Chinese Academy of Sciences and President of the Chinese Society of Biochemistry and Molecular Biology.\n\n## Biography", 'score': 0.25375047, 'raw_content': None}], 'response_time': 1.67, 'request_id': '55198a01-11b8-40ed-bd45-2ad774234b55'}), ToolMessage(content='[{"title": "Jingwen Peng, CFA - Director - Lead Data Steward at Liberty Mutual ...", "url": "https://www.linkedin.com/in/jingwen-peng-cfa-3a69b011?trk=public_profile_browsemap", "content": "Director - Lead Data Steward at Liberty Mutual Investments · Experience: Liberty Mutual Investments · Location: Boston · 500+ connections on LinkedIn.", "score": 0.70823324}, {"title": "Jingwen Peng - U of Rochester Simon STEM MSBA - LinkedIn", "url": "https://www.linkedin.com/in/jpeng19", "content": "●\\tAuthored product requirement documentation, followed up on new features and optimization based on customer feedback.\\n\\n●\\tDevised creative short videos content, elevated ad CTR by 4 times on Google Ads and refined data-driven strategies.\\n\\n●\\tGrew social media presence through influencer marketing, acquired 1,000+ new users on Instagram, FB, and YouTube. [...] At Amazon, my role as a Site Merchandising analyst was pivotal in enhancing the Kindle E-book business through data-driven strategies<br><br> By automating reports and refining forecasting models, our team significantly improved operational efficiency and influenced strategic decisions that positively impacted the growth of E-book store and Kindle Unlimited<br><br> My technical acumen, particularly in Excel, SQL and Python, was instrumental in automating processes, reducing manual workloads, [...] ## Experience\\n### Marketing Analyst  \\nEmpire City Casino  \\nMay 2025 - Present   \\nYonkers, New York, United States  \\n\\n### Amazon  \\nAmazon  \\nN/A - Present   \\nBeijing, China  \\n\\n### Marketing Analyst  \\nJiyuzhoutian Information Technology  \\nApr 2019 - Sep 2019   \\nBeijing, China  \\n●\\tSEO for website CouponBirds and ASO for app Spark, improved CTR by 50%, enhancing visibility and user acquisition.", "score": 0.6301622}, {"title": "[PDF] SCHWARZMAN SCHOLARS CLASS OF 2018", "url": "https://www.schwarzmanscholars.org/wp-content/uploads/2020/05/Schwarzman-Scholars-Class-of-2018-profiles-126-Scholars.pdf", "content": "Jingwen SUN graduated in May 2017 from the National University of Singapore with a Bachelor’s degree in Computing and Business. She is passionate about innovation and entrepreneurship. She has worked with venture capital funds in Israel and China and also worked with Accenture and Analysys Mason as an analyst intern. Jingwen is excited to join the Schwarzman Scholars program to learn more about leadership within a global context. Jingwen is 23 years old and from China. JONATHAN PADILLA UNITED [...] Assistant at Purdue, supporting both domestic and international students, and served as Vice President of Alpha Kappa Psi. She most recently worked in the Risk Assurance practice at PricewaterhouseCoopers in Chicago. At Schwarzman College, she aspires to amplify her leadership and business skills as both a humanitarian and an entrepreneur to improve and empower the education of youth in developing regions. Jing is 24 years old and from China. JINGWEN SUN CHINA NATIONAL UNIVERSITY OF SINGAPORE [...] over 10 villages in 4 provinces, interviewing over 600 people. He was appointed Academic Organizer of the China Youth Economic Forum and also served as the President of the Student Union of the School of Economics and the Chairman of the Standing Committee of the Students’ Congress. Peng is 22 years old and from China. PRESTON LIM CANADA PRINCETON UNIVERSITY Preston graduated in June 2017 from Princeton University with a major in Near Eastern Studies and a minor in History and the Practice of", "score": 0.38366494}, {"title": "Professor Peng GONG - GEOG | HKU", "url": "https://geog.hku.hk/p-gong", "content": "Guan, D.\\\\, Wang, D., Hallegatte, S. Steven J. Davis, Jingwen Huo, Shuping Li, Yangchun Bai, Tianyang Lei, Qianyu Xue, D’Maris Coffman, Danyang Cheng, Peipei Chen, Xi Liang, Bing Xu, Xiaosheng Lu, Shouyang Wang, Klaus Hubacek & Peng Gong. Global supply-chain effects of COVID-19 control measures. Nature Human Behaviour (2020). 4(6):577-587. . [...] Gong, P\\\\, Bin Chen, Xuecao Li, Han Liu, JieWang, Yuqi Bai, Jingming Chen, Xi Chen, Lei Fang, Shuailong Feng, Yongjiu Feng, Yali Gong, Hao Gu, Huabing Huang, Xiaochun Huang, Hongzan Jiao, Yingdong Kang, Guangbin Lei, Ainong Li, Xiaoting Li, Xun Li, Yuechen Li, Zhilin Li, Zhongde Li, Chong Liu, Chunxia Liu, Maochou Liu, Shuguang Liu, Wanliu Mao, Changhong Miao, Hao Ni, Qisheng Pan, Shuhua Qi, Zhehao Ren, Zhuoran Shan, Shaoqing Shen, Minjun Shi, Yimeng Song, Mo Su, HoiPing Suen, Bo Sun, Fangdi [...] Beijing Normal University, and the Tsinghua Urban Institute at Tsinghua University. In 2020, he led an expert preparation group in establishing the Vanke School of Public Health at Tsinghua.", "score": 0.30464175}, {"title": "Investigators | Center for Computational Biology and Bioinformatics", "url": "https://medicine.iu.edu/research-centers/computational-biology-bioinformatics/investigators", "content": "Associate Professor of Bioinformatics and Data Science\\n\\nRead Bio\\n\\n### Bohdan Khomtchouk, PhD\\n\\nAssistant Professor of Bioinformatics and Data Science\\n\\nRead Bio\\n\\n### Juexin Wang, PhD\\n\\nAssistant Professor of Bioinformatics\\n\\nRead Bio\\n\\n### Jingwen Yan, PhD\\n\\nAssistant Professor of Bioinformatics\\n\\nRead Bio\\n\\n### IU Bloomington Luddy School of Informatics, Computing and Engineering\\n\\n### Yijie Wang, PhD\\n\\nAssistant Professor of Computer Science\\n\\nRead Bio\\n\\n### Purdue University in Indianapolis [...] ### Jing Liu, PhD\\n\\nAssistant Professor of Physics\\n\\nRead Bio\\n\\n## Associate Members\\n\\nJunior computational scientists conducting research in quantitative biomedical sciences\\n\\n### Xiaoqing Huang, PhD\\n\\nAssistant Research Professor of Biostatistics & Health Data Science\\n\\nRead Bio\\n\\n### Gang Peng, PhD\\n\\nAssistant Professor of Medical & Molecular Genetics\\n\\nRead Bio\\n\\n### Jie Ren, PhD\\n\\nAssistant Professor of Biostatistics & Health Data Science\\n\\nRead Bio\\n\\n### Tae-Hwi L. Schwantes-An, PhD [...] Professor of Biochemistry & Molecular Biology\\n\\nRead Bio\\n\\n### Ian Webb, PhD\\n\\nAssistant Professor, Department of Chemistry & Chemical Biology\\n\\nRead Bio\\n\\n### Kai Yang, PhD\\n\\nAssociate Professor of Pediatrics\\n\\nRead Bio\\n\\n### Lei Yang, PhD\\n\\nProfessor of Pediatrics\\n\\nRead Bio\\n\\n### Ji Zhang, PhD\\n\\nAssociate Professor of Pediatrics\\n\\nRead Bio", "score": 0.19255488}]', name='tavily_search_results_json', id='2efa1c1e-58d6-4af2-8c57-ea2e8323e1c1', tool_call_id='call_KuvSnvV0DYPgSAFZMnxAqYLV', artifact={'query': 'Jingwen Peng', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.linkedin.com/in/jingwen-peng-cfa-3a69b011?trk=public_profile_browsemap', 'title': 'Jingwen Peng, CFA - Director - Lead Data Steward at Liberty Mutual ...', 'content': 'Director - Lead Data Steward at Liberty Mutual Investments · Experience: Liberty Mutual Investments · Location: Boston · 500+ connections on LinkedIn.', 'score': 0.70823324, 'raw_content': None}, {'url': 'https://www.linkedin.com/in/jpeng19', 'title': 'Jingwen Peng - U of Rochester Simon STEM MSBA - LinkedIn', 'content': '●\tAuthored product requirement documentation, followed up on new features and optimization based on customer feedback.\n\n●\tDevised creative short videos content, elevated ad CTR by 4 times on Google Ads and refined data-driven strategies.\n\n●\tGrew social media presence through influencer marketing, acquired 1,000+ new users on Instagram, FB, and YouTube. [...] At Amazon, my role as a Site Merchandising analyst was pivotal in enhancing the Kindle E-book business through data-driven strategies<br><br> By automating reports and refining forecasting models, our team significantly improved operational efficiency and influenced strategic decisions that positively impacted the growth of E-book store and Kindle Unlimited<br><br> My technical acumen, particularly in Excel, SQL and Python, was instrumental in automating processes, reducing manual workloads, [...] ## Experience\n### Marketing Analyst  \nEmpire City Casino  \nMay 2025 - Present   \nYonkers, New York, United States  \n\n### Amazon  \nAmazon  \nN/A - Present   \nBeijing, China  \n\n### Marketing Analyst  \nJiyuzhoutian Information Technology  \nApr 2019 - Sep 2019   \nBeijing, China  \n●\tSEO for website CouponBirds and ASO for app Spark, improved CTR by 50%, enhancing visibility and user acquisition.', 'score': 0.6301622, 'raw_content': None}, {'url': 'https://www.schwarzmanscholars.org/wp-content/uploads/2020/05/Schwarzman-Scholars-Class-of-2018-profiles-126-Scholars.pdf', 'title': '[PDF] SCHWARZMAN SCHOLARS CLASS OF 2018', 'content': 'Jingwen SUN graduated in May 2017 from the National University of Singapore with a Bachelor’s degree in Computing and Business. She is passionate about innovation and entrepreneurship. She has worked with venture capital funds in Israel and China and also worked with Accenture and Analysys Mason as an analyst intern. Jingwen is excited to join the Schwarzman Scholars program to learn more about leadership within a global context. Jingwen is 23 years old and from China. JONATHAN PADILLA UNITED [...] Assistant at Purdue, supporting both domestic and international students, and served as Vice President of Alpha Kappa Psi. She most recently worked in the Risk Assurance practice at PricewaterhouseCoopers in Chicago. At Schwarzman College, she aspires to amplify her leadership and business skills as both a humanitarian and an entrepreneur to improve and empower the education of youth in developing regions. Jing is 24 years old and from China. JINGWEN SUN CHINA NATIONAL UNIVERSITY OF SINGAPORE [...] over 10 villages in 4 provinces, interviewing over 600 people. He was appointed Academic Organizer of the China Youth Economic Forum and also served as the President of the Student Union of the School of Economics and the Chairman of the Standing Committee of the Students’ Congress. Peng is 22 years old and from China. PRESTON LIM CANADA PRINCETON UNIVERSITY Preston graduated in June 2017 from Princeton University with a major in Near Eastern Studies and a minor in History and the Practice of', 'score': 0.38366494, 'raw_content': None}, {'url': 'https://geog.hku.hk/p-gong', 'title': 'Professor Peng GONG - GEOG | HKU', 'content': 'Guan, D.\\, Wang, D., Hallegatte, S. Steven J. Davis, Jingwen Huo, Shuping Li, Yangchun Bai, Tianyang Lei, Qianyu Xue, D’Maris Coffman, Danyang Cheng, Peipei Chen, Xi Liang, Bing Xu, Xiaosheng Lu, Shouyang Wang, Klaus Hubacek & Peng Gong. Global supply-chain effects of COVID-19 control measures. Nature Human Behaviour (2020). 4(6):577-587. . [...] Gong, P\\, Bin Chen, Xuecao Li, Han Liu, JieWang, Yuqi Bai, Jingming Chen, Xi Chen, Lei Fang, Shuailong Feng, Yongjiu Feng, Yali Gong, Hao Gu, Huabing Huang, Xiaochun Huang, Hongzan Jiao, Yingdong Kang, Guangbin Lei, Ainong Li, Xiaoting Li, Xun Li, Yuechen Li, Zhilin Li, Zhongde Li, Chong Liu, Chunxia Liu, Maochou Liu, Shuguang Liu, Wanliu Mao, Changhong Miao, Hao Ni, Qisheng Pan, Shuhua Qi, Zhehao Ren, Zhuoran Shan, Shaoqing Shen, Minjun Shi, Yimeng Song, Mo Su, HoiPing Suen, Bo Sun, Fangdi [...] Beijing Normal University, and the Tsinghua Urban Institute at Tsinghua University. In 2020, he led an expert preparation group in establishing the Vanke School of Public Health at Tsinghua.', 'score': 0.30464175, 'raw_content': None}, {'url': 'https://medicine.iu.edu/research-centers/computational-biology-bioinformatics/investigators', 'title': 'Investigators | Center for Computational Biology and Bioinformatics', 'content': 'Associate Professor of Bioinformatics and Data Science\n\nRead Bio\n\n### Bohdan Khomtchouk, PhD\n\nAssistant Professor of Bioinformatics and Data Science\n\nRead Bio\n\n### Juexin Wang, PhD\n\nAssistant Professor of Bioinformatics\n\nRead Bio\n\n### Jingwen Yan, PhD\n\nAssistant Professor of Bioinformatics\n\nRead Bio\n\n### IU Bloomington Luddy School of Informatics, Computing and Engineering\n\n### Yijie Wang, PhD\n\nAssistant Professor of Computer Science\n\nRead Bio\n\n### Purdue University in Indianapolis [...] ### Jing Liu, PhD\n\nAssistant Professor of Physics\n\nRead Bio\n\n## Associate Members\n\nJunior computational scientists conducting research in quantitative biomedical sciences\n\n### Xiaoqing Huang, PhD\n\nAssistant Research Professor of Biostatistics & Health Data Science\n\nRead Bio\n\n### Gang Peng, PhD\n\nAssistant Professor of Medical & Molecular Genetics\n\nRead Bio\n\n### Jie Ren, PhD\n\nAssistant Professor of Biostatistics & Health Data Science\n\nRead Bio\n\n### Tae-Hwi L. Schwantes-An, PhD [...] Professor of Biochemistry & Molecular Biology\n\nRead Bio\n\n### Ian Webb, PhD\n\nAssistant Professor, Department of Chemistry & Chemical Biology\n\nRead Bio\n\n### Kai Yang, PhD\n\nAssociate Professor of Pediatrics\n\nRead Bio\n\n### Lei Yang, PhD\n\nProfessor of Pediatrics\n\nRead Bio\n\n### Ji Zhang, PhD\n\nAssociate Professor of Pediatrics\n\nRead Bio', 'score': 0.19255488, 'raw_content': None}], 'response_time': 1.63, 'request_id': '429125b7-11e4-4b4c-8cff-22a6734a27c4'})]



Receiving update from node: 'agent'
[AIMessage(content='Renjun Xu is currently a Principal Researcher at Zhejiang University and also associated with the Center for Data Science at Zhejiang University. Jingwen Peng is a Director and Lead Data Steward at Liberty Mutual Investments in Boston.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 6331, 'total_tokens': 6374, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2560}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ73IQm6GxBQWegjfZQiI2Czy1e7O', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--df884874-cc42-4bb4-ae70-588654a0f6be-0', usage_metadata={'input_tokens': 6331, 'output_tokens': 43, 'total_tokens': 6374, 'input_token_details': {'audio': 0, 'cache_read': 2560}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]



# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [27]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["text"])]}

def parse_output(input_state):
  return {"answer" : input_state["messages"][-1].content}

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

agent_chain_with_formatting.invoke({"text" : "What is Deep Research?"})

{'answer': 'Deep Research typically refers to an in-depth and comprehensive investigation or analysis into a specific topic, subject, or field. It involves thorough examination of available data, literature, and resources to uncover detailed insights, understand complex issues, and generate new knowledge or solutions. Deep Research is often characterized by rigorous methodology, critical thinking, and a focus on uncovering underlying principles or patterns.\n\nIf you are referring to a specific organization, product, or context named "Deep Research," please provide more details so I can give a more precise answer.'}

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    {
        "inputs" : {"text" : "Who were the main authors on the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' paper?"},
        "outputs" : {"must_mention" : ["Peng", "Xu"]}   
    },
    ...,
    {
        "inputs" : {"text" : "Where do the authors of the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' work now?"},
        "outputs" : {"must_mention" : ["Zhejiang", "Liberty Mutual"]}
    }
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions that pertain to the cohort use-case (more information [here](https://www.notion.so/Session-4-RAG-with-LangGraph-OSS-Local-Models-Eval-w-LangSmith-26acd547af3d80838d5beba464d7e701#26acd547af3d81d08809c9c82a462bdd)), or the use-case you're hoping to tackle in your Demo Day project.

In [28]:
questions = [
    {
        "inputs": {"text": "How do people use AI in their daily work?"},
        "outputs": {"must_mention": ["work", "productivity", "automation"]}
    },
    {
        "inputs": {"text": "What are the most common ways people use AI in their work?"},
        "outputs": {"must_mention": ["writing", "analysis", "coding", "automation"]}
    },
    {
        "inputs": {"text": "Do people use AI for their personal lives?"},
        "outputs": {"must_mention": ["personal", "learning", "entertainment", "planning"]}
    },
    {
        "inputs": {"text": "What concerns or challenges do people have when using AI?"},
        "outputs": {"must_mention": ["privacy", "accuracy", "bias", "job"]}
    },
    {
        "inputs": {"text": "What should I build to add value to the local communities I'm engaged in?"},
        "outputs": {"must_mention": ["community", "education", "accessibility", "local"]}
    },
    {
        "inputs": {"text": "What AI projects should companies build first to maximize ROI?"},
        "outputs": {"must_mention": ["ROI", "automation", "efficiency", "customer"]}
    },
    {
        "inputs": {"text": "How can AI help with customer service and support?"},
        "outputs": {"must_mention": ["customer", "support", "response", "automation"]}
    }
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [ ]:
from langsmith import Client

client = Client()

dataset_name = f"Simple Search Agent - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the cohort use-case to evaluate the Simple Search Agent."
)

client.create_examples(
    dataset_id=dataset.id,
    examples=questions
)

API key set: True


LangSmithError: Failed to POST /datasets in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets', '{"detail":"Forbidden"}')

### Task 2: Adding Evaluators

Let's use the OpenEvals library to product an evaluator that we can then pass into LangSmith!

> NOTE: Examine the `CORRECTNESS_PROMPT` below!

In [50]:
from openevals.prompts import CORRECTNESS_PROMPT
print(CORRECTNESS_PROMPT)

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

In [ ]:
from openevals.llm import create_llm_as_judge

correctness_evaluator = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT,
        model="openai:o3-mini", # very impactful to the final score
        feedback_key="correctness",
    )

Let's also create a custom Evaluator for our created dataset above - we do this by first making a simple Python function!

In [60]:
def must_mention(inputs: dict, outputs: dict, reference_outputs: dict) -> float:
  # determine if the phrases in the reference_outputs are in the outputs
  required = reference_outputs.get("must_mention") or []
  score = all(phrase in outputs["answer"] for phrase in required)
  return score

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [61]:
results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset.name,
    evaluators=[correctness_evaluator, must_mention],
    experiment_prefix="simple_agent, baseline",  # optional, experiment name prefix
    description="Testing the baseline system.",  # optional, experiment description
    max_concurrency=4, # optional, add concurrency
)

View the evaluation results for experiment: 'simple_agent, baseline-4085d794' at:
https://smith.langchain.com/o/340cd80b-3296-5752-9a9e-58582118073a/datasets/cd9ccfce-ebdd-4b2b-8c53-53df5a7a341b/compare?selectedSessions=6e4899d9-d28a-4008-8ffc-3e7883e9ef07




0it [00:00, ?it/s]

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [19]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #4:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

In [20]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE

In [21]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

##### YOUR MARKDOWN HERE

In [23]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

In [24]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

In [25]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

In [27]:
inputs = {"messages" : [HumanMessage(content="What are Deep Research Agents?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Deep Research Agents are advanced AI systems designed to assist with in-depth research tasks. They leverage deep learning techniques and large datasets to analyze complex information, generate insights, and support decision-making across various fields such as science, technology, medicine, and more. These agents can automate literature reviews, extract relevant data from vast sources, and provide comprehensive summaries, making research more efficient and thorough. Would you like me to find more detailed or specific information about Deep Research Agents?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 158, 'total_tokens': 251, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-

## Part 3: LangGraph for the "Patterns" of GenAI

### Task 4: Helpfulness Check of Gen AI Pattern Descriptions

Let's ask our system about the 3 main patterns in Generative AI:

1. Context Engineering
2. Fine-tuning
3. Agents

In [28]:
patterns = ["Context Engineering", "Fine-tuning", "LLM-based agents"]

In [29]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Context Engineering is a relatively new interdisciplinary field that focuses on designing, managing, and optimizing the context in which systems, especially artificial intelligence and software applications, operate. It involves understanding and shaping the environment, circumstances, and background information that influence how systems behave and interact with users. The goal is to improve system performance, user experience, and decision-making by carefully engineering the context.

The concept of Context Engineering has gained prominence with the rise of AI, IoT, and complex software systems, where context plays a crucial role in system effectiveness. It started to break onto the scene in the late 2010s and early 2020s, driven by advancements in contextual AI, ubiquitous computing, and the need for more adaptive and personalized systems.

Would you like me to find more detailed or specific information about its origins and development?



Fine-tuning is a machine learning techniqu